In [2]:
#importar as bibliotecas
import pandas as pd

#importar base de dados
emails = pd.read_excel(r"Bases de Dados\Emails.xlsx")
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [3]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')

In [4]:
dicionario_lojas={}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

In [5]:
dia_indicador = vendas['Data'].max()

In [24]:
import pathlib

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

In [17]:
loja = "Norte Shopping"
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

# faturamento
faturamento_ano = vendas_loja_dia['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()

# diversidade de produtos
qtde_produtos_ano = len(vendas_loja['Produto'].unique())
qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

# tiket medio
valor_venda = vendas_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()



TypeError: datetime64 type does not support sum operations